In [ ]:
from pipeline.datasets.sevir.sevir import SEVIRLightningDataModule
import time
from omegaconf import OmegaConf
cfg = OmegaConf.load("/home/vatsal/NWM/weatherforecasting/pipeline/datasets/sevir/config.yaml").Dataset
dm = SEVIRLightningDataModule(cfg)
dm.prepare_data()
setup_s = time.time()
dm.setup()
setup_e = time.time()
print(f"took : {setup_e - setup_s}")

l_s = time.time()
loader = dm.train_dataloader()
idx = 0
for sample in loader:
    idx += 1
    if idx >= 200:break
l_e = time.time()
print("loading", l_e - l_s)
l_train_full = len(dm.train_dataloader())
l_val_full = len(dm.val_dataloader())
l_test_full = len(dm.test_dataloader())
print(l_train_full, l_val_full, l_test_full)


In [ ]:
from pipeline.datasets.sevir.sevir import SEVIRLightningDataModule
import time
from omegaconf import OmegaConf
cfg = OmegaConf.load("/home/vatsal/NWM/weatherforecasting/pipeline/datasets/sevir/fast_config.yaml").Dataset
dm = SEVIRLightningDataModule(cfg)
dm.prepare_data()
setup_s = time.time()
dm.setup()
setup_e = time.time()
print(f"took : {setup_e - setup_s}")

l_s = time.time()
loader = dm.train_dataloader()
idx = 0
for sample in loader:
    idx += 1
    if idx >= 200:break
l_e = time.time()
print("loading", l_e - l_s)
l_train_fast = len(dm.train_dataloader())
l_val_fast = len(dm.val_dataloader())
l_test_fast = len(dm.test_dataloader())
print(l_train_fast, l_val_fast, l_test_fast)

In [ ]:
print(l_train_fast/l_train_full)
print(l_val_fast/l_val_full)
print(l_test_fast/l_test_full)

In [ ]:
dummpy = torch.randn(8, 52, 2304)
from omegaconf import OmegaConf
cfg = OmegaConf.load("/home/vatsal/NWM/weatherforecasting/experiments/pretrained_ae_dlinear_indc_indp/config.yaml").dlinear
model = DLinear(cfg)
model2 = DLinear2(cfg)
model.train()
model2.train()
state_dict = model.state_dict()
model2.load_state_dict(state_dict)
out1 = model(dummpy)
out2 = model2(dummpy)
#check all parameters are same
flag = True
for p1, p2 in zip(model.parameters(), model2.parameters()):
    if not torch.allclose(p1, p2, atol=1e-4):
        flag = False
        break
print("Parameters match:", flag)
print(out1.shape, out2.shape)
print(torch.allclose(out1, out2, atol=1e-12))  #
target = torch.randn(8, 48, 2304) 
loss1 = F.mse_loss(out1, target)
loss2 = F.mse_loss(out2, target)

loss1.backward()
loss2.backward()
optim = torch.optim.SGD(model.parameters(), lr=1e-3)
optim2 = torch.optim.SGD(model2.parameters(), lr=1e-3)
optim.step()
optim2.step()

# Check if the gradients are the same
flag = True
for (name1, p1), (name2, p2) in zip(model.named_parameters(), model2.named_parameters()):
    if p1.grad is None or p2.grad is None:
        print(f"Gradient is None for: {name1}, {name2}")
        assert p1.grad is None and p2.grad is None, "both gradients should be None"
        print("skipping gradient check for None gradients")
        continue
    if not torch.allclose(p1.grad, p2.grad, atol=1e-4):
        flag = False
        break
print("Gradients match:", flag)

out1_again = model(dummpy)
out2_again = model2(dummpy)
print(torch.allclose(out1_again, out2_again, atol=1e-12))  # should be True after training

In [ ]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_ch)
        self.act1 = nn.GELU()
        
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_ch)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_ch)
            )
            
        self.act2 = nn.GELU()

    def forward(self, x):
        shortcut = self.shortcut(x)
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.act1(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        out += shortcut
        return self.act2(out)

class UpsampleBlock(nn.Module):
    def __init__(self, in_ch, out_ch, scale_factor):
        super().__init__()
        self.upsample = nn.Upsample(scale_factor=scale_factor, mode='nearest')
        self.resblock = ResidualBlock(in_ch, out_ch, stride=1)

    def forward(self, x):
        return self.resblock(self.upsample(x))

class ConvAutoencoder(nn.Module):
    def __init__(self, in_ch=1, latent_dim=512):
        super().__init__()
        
        # Encoder: 128 → 64 → 32 → 16 → 4 → 1
        self.enc1 = ResidualBlock(in_ch,   64, stride=2)  # 128 → 64
        self.enc2 = ResidualBlock(64,     128, stride=2)  # 64 → 32
        self.enc3 = ResidualBlock(128,    256, stride=2)  # 32 → 16
        self.enc4 = ResidualBlock(256,    512, stride=4)  # 16 → 4
        self.enc5 = ResidualBlock(512,   1024, stride=4)  # 4 → 1

        self.flatten = nn.Flatten()                     # (B,1024,1,1) → (B,1024)
        self.fc_enc = nn.Linear(1024, latent_dim)

        self.fc_dec = nn.Linear(latent_dim, 1024)
        self.unflatten = nn.Unflatten(1, (1024, 1, 1))   # (B,1024) → (B,1024,1,1)
        self.dec_init_conv = ResidualBlock(1024, 1024, stride=1)

        # Decoder: 1 → 4 → 16 → 32 → 64 → 128
        self.dec1 = UpsampleBlock(1024, 512, scale_factor=4)  # 1 → 4
        self.dec2 = UpsampleBlock(512,  256, scale_factor=4)  # 4 → 16
        self.dec3 = UpsampleBlock(256,  128, scale_factor=2)  # 16 → 32
        self.dec4 = UpsampleBlock(128,   64, scale_factor=2)  # 32 → 64
        
        self.final_upsample = nn.Upsample(scale_factor=2, mode='nearest') # 64 → 128
        self.final_conv = nn.Conv2d(64, in_ch, kernel_size=3, stride=1, padding=1)

    def encode(self, x):
        x = self.enc1(x)
        x = self.enc2(x)
        x = self.enc3(x)
        x = self.enc4(x)
        x = self.enc5(x)
        x = self.flatten(x)
        z = self.fc_enc(x)
        return z

    def decode(self, z):
        x = self.fc_dec(z)
        x = self.unflatten(x)
        x = self.dec_init_conv(x)
        x = self.dec1(x)
        x = self.dec2(x)
        x = self.dec3(x)
        x = self.dec4(x)
        x = self.final_upsample(x)
        x = self.final_conv(x)
        return x

    def forward(self, x):
        z = self.encode(x)
        reconstruction = self.decode(z)
        return reconstruction, z

if __name__ == "__main__":
    import torch
    model = ConvAutoencoder()
    x = torch.randn(8, 1, 128, 128)  # Batch of 8 images with 1 channel, 128x128 size
    recon, z = model(x)
    print("Input shape:", x.shape)
    print("Reconstruction shape:", recon.shape)
    print("Latent vector shape:", z.shape)
    params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    params_in_million = params / 1e6
    print(f"Total trainable parameters: {params_in_million:.2f} million")

In [ ]:
path = "/home/vatsal/NWM/sevir_lr/data/vil/2017/SEVIR_VIL_RANDOMEVENTS_2017_0501_0831.h5"
import h5py
data = h5py.File(path, 'r')
vil = data['vil'][:]
print("Shape of VIL data:", vil.shape)

In [ ]:
import datetime
import os
os.environ['WANDB_API_KEY'] = '041eda3850f131617ee1d1c9714e6230c6ac4772'
datetime.datetime.now().strftime("_%Y%m%d_%H%M%S")
from pytorch_lightning.loggers import WandbLogger
exp_name = "test_run_xyz"
save_dir = os.path.join("experiments", "test", exp_name + datetime.datetime.now().strftime("_%Y%m%d_%H%M%S"))
version = "v1.0"
logger = WandbLogger(project="delete_it", name="test_run", save_dir=save_dir, version=version)

In [ ]:
from pipeline.datasets.sevire.sevir import SEVIRLightningDataModule
data = SEVIRLightningDataModule(dataset_name="sevir_lr", num_workers=0, batch_size=1, seq_len=1, stride=1, layout='NTHW')
data.setup()
data.prepare_data()
train_loader = data.train_dataloader()
sample1 = next(iter(train_loader))

In [ ]:
len(train_loader)

In [ ]:
from pipeline.datasets.sevire.sevir import SEVIRLightningDataModule
data2 = SEVIRLightningDataModule(dataset_name="sevir_lr", num_workers=8, batch_size=8, seq_len=12, stride=20, layout='NTHW')
data2.setup()
data2.prepare_data()
train_loader2 = data2.train_dataloader()
sample2 = next(iter(train_loader2))

In [ ]:
data2_dict = {}
print(len(train_loader2))
from tqdm import tqdm
for epoch in range(2):
    print(f"Epoch {epoch+1}")
    if epoch == 0:
        for idx, sample in tqdm(enumerate(train_loader2)):
            data = sample['vil']
            data2_dict[data] = idx
    else:
        for sample in tqdm(train_loader2):
            data = sample['vil']
            idx = -1
            for i, d in enumerate(data2_dict.keys()):
                if torch.allclose(d, data, atol=1e-12):
                    idx = i
                    break
            print(f"Found data at index: {idx}")

In [ ]:
for idx, sample in tqdm(enumerate(train_loader2)):
    

In [ ]:
from pipeline.datasets.sevire.sevir import SEVIRLightningDataModule
data3 = SEVIRLightningDataModule(dataset_name="sevir_lr", num_workers=0, batch_size=8, seq_len=1, stride=1, layout='NTHW')
data3.setup()
data3.prepare_data()
train_loader3 = data2.train_dataloader()

In [ ]:
for sample2, sample3 in zip(train_loader2, train_loader3):
    data2 = sample2['vil']
    data3 = sample3['vil']
    if not torch.allclose(data2, data3, atol=1e-12):
        print("Data mismatch found!")
        break

In [ ]:
sample3 = next(iter(train_loader3))
import torch
torch.allclose(sample2['vil'], sample3['vil'], atol=1e-12) 

In [ ]:
datas = []
import torch
for sample in train_loader2:
    data = sample['vil']
    datas.append(data)
print(len(datas))
for sample in train_loader3:
    data = sample['vil']
    idx = -1
    for i, d in enumerate(datas):
        if torch.allclose(data, d, atol=1e-12):
            idx = i
            break
    if idx != -1:
        print(f"Found matching data at index {idx}")
    else:
        print("No matching data found _______________________________-")
        break

In [ ]:
datas = []
import torch
for sample in train_loader2:
    data = sample['vil']
    datas.append(data)
print(len(datas))
for sample in train_loader2:
    data = sample['vil']
    idx = -1
    for i, d in enumerate(datas):
        if torch.allclose(data, d, atol=1e-12):
            idx = i
            break
    if idx != -1:
        print(f"Found matching data at index {idx}")
    else:
        print("No matching data found _______________________________-")
        break

In [ ]:
datas = []
import torch
for sample in train_loader2:
    data = sample['vil']
    for bdata in data:
        datas.append(bdata)
print(len(datas))
for sample in train_loader3:
    data = sample['vil']
    for bdata in data:
        idx = -1
        for i, d in enumerate(datas):
            if torch.allclose(bdata, d, atol=1e-12):
                idx = i
                break
        if idx != -1:
            print(f"Found matching data at index {idx}")
        else:
            print("No matching data found _______________________________-")
            break

In [ ]:
import os
import time
import glob

def cleanup_checkpoints_and_wandb(root_dir):
    total_freed = 0
    # Delete .ckpt files (keep most recent)
    for dirpath, dirnames, _ in os.walk(root_dir):
        if os.path.basename(dirpath) == "checkpoints":
            ckpts = glob.glob(os.path.join(dirpath, "*.ckpt"))
            if len(ckpts) <= 1:
                continue
            ckpts = [(f, os.path.getmtime(f)) for f in ckpts]
            ckpts.sort(key=lambda x: x[1], reverse=True)
            to_delete = ckpts[1:]  # keep most recent

            for fpath, _ in to_delete:
                try:
                    size_mb = os.path.getsize(fpath) / (1024 * 1024)
                    os.remove(fpath)
                    total_freed += size_mb
                    print(f"Deleted .ckpt: {fpath} ({size_mb:.2f} MB)")
                except Exception as e:
                    print(f"Failed to delete {fpath}: {e}")

    # Delete all .wandb files
    for dirpath, _, filenames in os.walk(root_dir):
        for fname in filenames:
            if fname.endswith(".wandb"):
                fpath = os.path.join(dirpath, fname)
                try:
                    size_mb = os.path.getsize(fpath) / (1024 * 1024)
                    os.remove(fpath)
                    total_freed += size_mb
                    print(f"Deleted .wandb: {fpath} ({size_mb:.2f} MB)")
                except Exception as e:
                    print(f"Failed to delete {fpath}: {e}")

    print(f"Total space freed: {total_freed:.2f} MB")

if __name__ == "__main__":
    cleanup_checkpoints_and_wandb("/home/vatsal/NWM/weatherforecasting")


In [ ]:
import os

def get_total_png_size(root_dir):
    total_size = 0
    for dirpath, _, filenames in os.walk(root_dir):
        for fname in filenames:
            if fname.endswith(".png"):
                fpath = os.path.join(dirpath, fname)
                try:
                    total_size += os.path.getsize(fpath)
                except Exception as e:
                    print(f"Error reading {fpath}: {e}")
    print(f"Total .png size: {total_size / (1024 * 1024):.2f} MB")

if __name__ == "__main__":
    get_total_png_size("/home/vatsal/NWM/weatherforecasting/experiments/pretrained_ae_linear_sevir/outputs")


In [ ]:
import os

def get_ckpt_total_size(root_dir):
    total_size = 0
    for dirpath, _, filenames in os.walk(root_dir):
        for fname in filenames:
            if fname.endswith('.ckpt'):
                fpath = os.path.join(dirpath, fname)
                total_size += os.path.getsize(fpath)
    return total_size

def format_size(bytes_size):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if bytes_size < 1024:
            return f"{bytes_size:.2f} {unit}"
        bytes_size /= 1024
    return f"{bytes_size:.2f} PB"

root = "/home/vatsal/NWM"
total = get_ckpt_total_size(root)
print("Total .ckpt size:", format_size(total))


Total .ckpt size: 5.23 GB


In [31]:
import os

def get_dir_size(path):
    total = 0
    for dirpath, _, filenames in os.walk(path):
        for f in filenames:
            fpath = os.path.join(dirpath, f)
            try:
                if not os.path.islink(fpath) and os.path.isfile(fpath):
                    total += os.path.getsize(fpath)
            except:
                continue
    return total

def list_top_dirs(root_dir, top_n=20):
    dir_sizes = []
    for entry in os.scandir(root_dir):
        if entry.is_dir(follow_symlinks=False):
            size = get_dir_size(entry.path)
            dir_sizes.append((entry.path, size))
    
    dir_sizes.sort(key=lambda x: x[1], reverse=True)
    for path, size in dir_sizes[:top_n]:
        print(f"{path}: {size / (1024**3):.2f} GB")

# Example usage
list_top_dirs("/home/vatsal/NWM", top_n=20)


/home/vatsal/NWM/weatherforecasting: 20.60 GB
/home/vatsal/NWM/weather: 11.77 GB
/home/vatsal/NWM/sevir_lr: 8.77 GB
/home/vatsal/NWM/PreDiff: 0.36 GB
/home/vatsal/NWM/DiffCast: 0.16 GB
/home/vatsal/NWM/AlphaPre: 0.00 GB
/home/vatsal/NWM/.vscode: 0.00 GB
/home/vatsal/NWM/wandb: 0.00 GB
